In [4]:
import tweepy
from dotenv import load_dotenv
import os
import pandas as pd
from pymongo import MongoClient
# from google.cloud import bigquery
import json

client = MongoClient('localhost', 27017)

# os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
# bq_client = bigquery.Client()

db = client.worldcup

load_dotenv()

tweepy_client = tweepy.Client(os.getenv('BEARER_TOKEN'), wait_on_rate_limit=True)

TypeError: str expected, not NoneType

In [105]:
query = '#ARG OR #HRV -is:retweet'

paginator = tweepy.Paginator(tweepy_client.search_recent_tweets, 
                        query=query,
                        tweet_fields = ['author_id', 'created_at', 'source', 'lang', 'geo', 'public_metrics', 'entities', 'context_annotations', 'attachments'], 
                        media_fields = ['preview_image_url', 'media_key', 'type', 'url'],
                        exclude=['retweets', 'replies'],
                        expansions = ['author_id', 'attachments.media_keys'], 
                        start_time = '2022-12-13T00:00:00.000Z',
                        end_time = '2022-12-14T00:00:00.000Z',
                        max_results = 100
                        )

for page in paginator:
    
    users = {u["id"]: u for u in page.includes['users']}
    media = {m["media_key"]: m for m in page.includes['media']}

    for tweet in page.data:

        if users[tweet.author_id]:
            user = users[tweet.author_id]

        # if media:
        #     attachments = tweet.data['attachments']
        #     media_keys = attachments['media_keys']
        #     media_info = {
        #         "media_key": media_keys,
        #         "media_type": media[media_keys[0]].type,
        #         "preview_image_url": media[media_keys[0]].preview_image_url
        #     }
        # else:
        #     media_info = ""
        
        tweet_content = {
            "tweet_id": tweet.id,
            "author": user,
            "created_at": tweet.created_at,
            "source": tweet.source,
            "lang": tweet.lang,
            "geo": tweet.geo,
            "public_metrics": tweet.public_metrics,
            "entites": tweet.entities,
            "text": tweet.text,
            # "media": media_info
        }
        
        db.testarghrv.insert_one(tweet_content)

# for tweet in tweets.flatten():
#     try:
#         print("1. ",tweet.data['entities'])
#         print("2. ",tweet.includes['name'])
#         for key, value in tweet.data.items():
#             print (key, value)
#         # db.a.insert_one(tweet)
#         break
#     except ValueError as e:
#         print(e)

In [3]:
arghrv = db.testarghrv.find()

for ah in arghrv:
    print(ah)
    hashtags = []
    for hashtag in ah.entities.hashtag:
        hashtags.append(hashtag.tag)
    tweet = {
        "tweet_id": ah.tweet_id,
        "author_id": ah.author.id,
        "author_name": ah.author.name,
        "author_username": ah.author.username,
        "created_at": ah.created_at,
        "source": ah.source,
        "lang": ah.lang,
        "geo": ah.geo,
        "retweet": ah.retweet.retweet_count,
        "reply": ah.retweet.reply_count,
        "like": ah.retweet.like_count,
        "quote": ah.retweet.quote_count,
        "hashtags": ",".join(hashtags),
        "mentions": ah.entities,
        "hashtags": ah.entities,
        "hashtags": ah.entities,
        "text": tweet.text,
    }
    break

{'_id': ObjectId('63991486c3841c0557044353'), 'tweet_id': 1602815620353073152, 'author': {'id': 1152570459835252737, 'name': 'Cucciolina9625', 'username': 'Cucciolina96251'}, 'created_at': datetime.datetime(2022, 12, 13, 23, 59, 57), 'source': 'Twitter for Android', 'lang': 'it', 'geo': None, 'retweet': {'retweet_count': 1, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}, 'entites': {'hashtags': [{'start': 17, 'end': 26, 'tag': 'Mondiale'}, {'start': 66, 'end': 70, 'tag': 'ARG'}, {'start': 71, 'end': 75, 'tag': 'HRV'}, {'start': 76, 'end': 86, 'tag': 'Qatar2022'}], 'mentions': [{'start': 3, 'end': 15, 'username': 'corradone91', 'id': '757599085641789440'}]}, 'text': 'RT @corradone91: #Mondiale onnipotente di Leo Messi. STREPITOSO \n\n#ARG #HRV #Qatar2022'}


In [ ]:
jpn_vs_hrv = bq_client.get_table('tecky-capstone-project.worldcup.jpn_hrv')

x = db.jh.find()
for row in x:
    row['_id'] = str(row['_id'])
    row['Date_Created'] = str(row['Date_Created'])
    bq_client.insert_rows_json(jpn_vs_hrv,[row])